In [36]:
#setup
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
from mpl_toolkits import mplot3d
plotly.tools.set_credentials_file(username='xavier2696', api_key='97cuXMXYTejnMxo9gpIk')
import re
from sklearn import preprocessing, metrics, decomposition, pipeline, dummy
import math 
import sys

In [47]:
buildings_df = pd.read_pickle('data/building/tainan/buildings_with_everything.pkl')

In [48]:
buildings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193559 entries, 0 to 193558
Data columns (total 16 columns):
base_area                193559 non-null float64
building_area            193559 non-null float64
category                 193559 non-null object
construction_type        193559 non-null object
date                     193559 non-null object
height                   193559 non-null float64
latitude                 193559 non-null float64
license_category         193559 non-null object
longitude                193559 non-null float64
number_basements         193559 non-null int64
number_design_changes    193559 non-null int64
number_floors            193559 non-null int64
number_households        193559 non-null int64
total_floor_area         193559 non-null float64
damage_level             193559 non-null int64
liquefaction             193559 non-null int64
dtypes: float64(6), int64(6), object(4)
memory usage: 23.6+ MB


In [49]:
buildings_df.category.value_counts()

old    193559
Name: category, dtype: int64

In [39]:
buildings_df = buildings_df[buildings_df.liquefaction != -1]

In [40]:
buildings_df.liquefaction.value_counts()

1    173155
0       823
Name: liquefaction, dtype: int64

In [41]:
earthquakes_df = pd.read_pickle('data/earthquake/earthquakes_Tainan_withData_withPeakValue_dropUseless_removeDuplicate.pkl')

In [42]:
earthquakes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 665 entries, 0 to 664
Data columns (total 28 columns):
index                  665 non-null int64
amplitude_unit         665 non-null object
depth                  665 non-null float64
ea                     665 non-null object
epicenter_latitude     665 non-null float64
epicenter_longitude    665 non-null float64
instrument_kind        665 non-null object
magnitude              665 non-null float64
na                     665 non-null object
origin_time            665 non-null object
record_length          665 non-null float64
sample_rate            665 non-null float64
start_time             665 non-null object
station_code           665 non-null object
station_latitude       665 non-null float64
station_longitude      665 non-null float64
station_name           665 non-null object
time                   665 non-null object
ua                     665 non-null object
e_PGA                  665 non-null float64
e_PGV                  665 

In [43]:
def get_earthquake_data(lat, lon, df):
    min_difference = sys.float_info.max
    earthquake_data = [0,0,0,0,0,0,0,0,0,0,0]
    for index, row in df.iterrows():
        difference = math.sqrt(math.pow(abs(row.station_latitude - lat), 2) + math.pow(abs(row.station_longitude - lon), 2))
        if difference < min_difference:
            min_difference = difference
            earthquake_data = [row.e_PGA, row.e_PGV, row.e_PGD, row.n_PGA, row.n_PGV, row.n_PGD, row.u_PGA, row.u_PGV, row.u_PGD, row.station_latitude, row.station_longitude]
    return earthquake_data

In [44]:
#add closest station data to buildings_df
e_PGA = []
e_PGV = []
e_PGD = []
n_PGA = []
n_PGV = []
n_PGD = []
u_PGA = []
u_PGV = []
u_PGD = []
for index, row in buildings_df.iterrows():
    e_data = get_earthquake_data(row.latitude, row.longitude, earthquakes_df)
    e_PGA += [e_data[0]]
    e_PGV += [e_data[1]]
    e_PGD += [e_data[2]]
    n_PGA += [e_data[3]]
    n_PGV += [e_data[4]]
    n_PGD += [e_data[5]]
    u_PGA += [e_data[6]]
    u_PGV += [e_data[7]]
    u_PGD += [e_data[8]]
    
buildings_df['e_PGA'] = e_PGA
buildings_df['e_PGV'] = e_PGV
buildings_df['e_PGD'] = e_PGD
buildings_df['n_PGA'] = n_PGA
buildings_df['n_PGV'] = n_PGV
buildings_df['n_PGD'] = n_PGD
buildings_df['u_PGA'] = u_PGA
buildings_df['u_PGV'] = u_PGV
buildings_df['u_PGD'] = u_PGD
    

In [45]:
buildings_df[0:10]

,base_area,building_area,category,construction_type,date,height,latitude,license_category,longitude,number_basements,...,liquefaction,e_PGA,e_PGV,e_PGD,n_PGA,n_PGV,n_PGD,u_PGA,u_PGV,u_PGD
1,0.0,0.0,old,unknown,年月日,7.0,22.96486,use,120.229242,0,...,1,4.90862,1.763957,0.633982,4.432334,1.80962,1.023354,1.957832,0.92276,0.443129
2,0.0,0.0,old,unknown,年月日,7.0,22.96486,use,120.229242,0,...,1,4.90862,1.763957,0.633982,4.432334,1.80962,1.023354,1.957832,0.92276,0.443129
3,0.0,0.0,old,unknown,年月日,7.0,22.96486,use,120.229242,0,...,1,4.90862,1.763957,0.633982,4.432334,1.80962,1.023354,1.957832,0.92276,0.443129
4,0.0,0.0,old,unknown,年月日,7.0,22.96486,use,120.229242,0,...,1,4.90862,1.763957,0.633982,4.432334,1.80962,1.023354,1.957832,0.92276,0.443129
5,0.0,0.0,old,unknown,年月日,7.0,22.96486,use,120.229242,0,...,1,4.90862,1.763957,0.633982,4.432334,1.80962,1.023354,1.957832,0.92276,0.443129
6,0.0,0.0,old,unknown,年月日,7.0,22.96486,use,120.229242,0,...,1,4.90862,1.763957,0.633982,4.432334,1.80962,1.023354,1.957832,0.92276,0.443129
7,0.0,0.0,old,unknown,年月日,7.0,22.96486,use,120.229242,0,...,1,4.90862,1.763957,0.633982,4.432334,1.80962,1.023354,1.957832,0.92276,0.443129
8,0.0,0.0,old,unknown,年月日,7.0,22.96486,use,120.229242,0,...,1,4.90862,1.763957,0.633982,4.432334,1.80962,1.023354,1.957832,0.92276,0.443129
9,0.0,0.0,old,unknown,年月日,7.0,22.96486,use,120.229242,0,...,1,4.90862,1.763957,0.633982,4.432334,1.80962,1.023354,1.957832,0.92276,0.443129
10,0.0,0.0,old,unknown,年月日,7.0,22.96486,use,120.229242,0,...,1,4.90862,1.763957,0.633982,4.432334,1.80962,1.023354,1.957832,0.92276,0.443129


In [46]:
buildings_df.to_pickle('all_tainan_data.pkl')